# Introduction to Linear Optimization in Python

Linear optimization (also known as linear programming) is a powerful mathematical technique used to find the best outcome in a mathematical model whose requirements are represented by linear relationships. It's a cornerstone of operations research that helps organizations make optimal decisions when faced with complex constraints and competing objectives.

As a data scientist or analyst, you've likely encountered problems where you need to maximize or minimize an outcome subject to various constraints. Whether it's optimizing a supply chain, allocating resources, or planning production schedules, linear optimization provides a structured approach to finding optimal solutions.

In this notebook, we'll explore how to formulate and solve linear optimization problems using Pyomo, an open-source Python-based optimization modeling language. Pyomo offers a flexible and intuitive way to express optimization problems and connects seamlessly with various solvers. 

The example that we will use to motivate our exploration comes from the petroleum industry. There are hundreds of different grades of crude oil in the world, each with different chemical properties. Oil refineries are configured to run using grades whose chemical properties fall within certain bounds. Often, raw crudes from different sources are not suitable for direct use in the refinery, but by blending different grades in the right ratios, a crude oil blend with suitable chemical properties can be created for use in the refinery. 

#### What You Will Learn

* The essential parts of a linear optimization model
* How to translate the problem statement into math and into code
* How to use basic algebra to convert a non-linear problem into a linear one
* How to program variables and constraints that are integers
* How to parse the solution results


In [ ]:
%cd /app

import json
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from utils.blend_indices import blend_indices

solver = pyo.SolverFactory('glpk')

In [ ]:
with open("data/assay.json") as f:
    assay = json.load(f)

CONSTRAINTS = {
    "ibp220": [0.0, 19.8],
    "htsd50": [470, 570],
    "1020plus": [0, 16],
    "v": [0,15],
    "ni": [0.0, 8.0],
    "neut": [0.0, 0.28],
    "mcrt": [0.0, 2.4],
    "vis100f": [0.21, 60], # set to be 0.2 + e, e>0
    "rvp": [0.0, 9.5],
    "api": [37, 42],
    "sulf":[0.0, 0.4185],
    "pour": [-50,50]
}

SUPPLY = {
    'crude_1': [0, 300000],
    'crude_2': [0, 300000],
    'crude_3': [0, 300000],
    'crude_4': [0, 300000],
    'crude_5': [0, 300000],
    'crude_6': [0, 300000],
    'crude_7': [0, 300000],
    'crude_8': [0, 300000],
    'crude_9': [0, 300000],
    'crude_10': [0, 300000]
}

TOTAL = 500000
MAX_CRUDES = 3



index_constraints = {}

for q, r in CONSTRAINTS.items():
    if q in blend_indices:

        q_idx = blend_indices[q]['index_name']
        func = blend_indices[q]['func']

        if q == "api":
            index_constraints[q_idx] = [func(r[1]), func(r[0])]
        else:
            index_constraints[q_idx] = [func(r[0]), func(r[1])]
    else:
        index_constraints[q] = r


index_assay = {}

for crude, quality_dict in assay.items():
    index_assay[crude] = {}
    for q, v in quality_dict.items():
        if q in blend_indices:
            q_idx = blend_indices[q]['index_name']
            func = blend_indices[q]['func']

            index_assay[crude][q_idx] = func(v)
        else:
            index_assay[crude][q] = v



constraints = {
    'quality': index_constraints,
    'supply': SUPPLY,
    'total': TOTAL,
    'max_crudes': MAX_CRUDES
}


prices = {}
for c, p in zip(assay.keys(), np.random.uniform(low=65,high=75, size=10)):
    prices[c] = p


# Model Formulation 

Linear optimization problems have four key components: *parameters*, *variables*, *constraints* and *sets*. 

1. **Parameters** define fixed data in the model
2. **Variables** are the quantities to be optimized
3. **Constraints** define the space of possible solutions
4. **Sets** are used to index parameters, variables and constraints
5. **Objective Function** is used to define the 


## Sets

Sets are used to index over parameters, variables and constraints. 

1. *grades* of crude (denoted $C$) and 
2. chemical *qualities* (denoted $Q$) which are different chemical properties of a grade of crude oil

In [ ]:
## Define Sets ###
model = pyo.ConcreteModel()

model.CRUDES = pyo.Set(initialize=[c for c in index_assay.keys()])
model.QUALITIES = pyo.Set(initialize=[q for q in constraints['quality'].keys()])

model.CRUDES.pprint()
model.QUALITIES.pprint()

## Variables and Parameters

### Parameters

The blend model has two sets of parameters, the qualities of individual crude grades and the prices of each grade. For example, each grade will have its own API gravity, sulfer content and viscosity. Each grade will also have its own price. Denote

1. the value of quality $j \in Q$ for crude $i \in C$ as $q_{ij}$
2. the price of crude $i \in C$ as $p_i$

The prices and qualities are fixed quantities of the model and are initialized from data. 

In [ ]:
### Define Parameters ###
def initialize_coefficients(model, crude, quality):
    return float(index_assay[crude][quality])

model.qualities = pyo.Param(model.CRUDES * model.QUALITIES, initialize=initialize_coefficients, domain=pyo.Reals)

def initialize_prices(model, crude):
    return prices[crude]

model.prices = pyo.Param(model.CRUDES, initialize=initialize_prices, domain=pyo.Reals)

### Variables

Let $x_i$ denote the *quantity* of crude $i \in C$ (measured in barrels). These variables are what the model solves for. We also place constraints on the values each variable can take based on the type of problem we are solving. Since we cannot purchas a negative amount of crude, we restrict $x_i$ to be non-negative

$$
x_i \geq 0
$$

We also allow the user to set a custom supply of each crude, denoted $U_i$. We can optionally also force the model to use a minimum amount of crude, denoted $L_i$. Together, these constraints combine to 

$$
0 \leq L_i \leq x_i \leq U_i \ \ \ \text{for all} \ \ i \in C
$$

In [ ]:
def initialize_bounds(model, i):
    return (constraints['supply'][i][0], constraints['supply'][i][1])

model.x = pyo.Var(model.CRUDES, domain=pyo.NonNegativeReals, bounds=initialize_bounds)

## Constraints

Constraints are imposed on the set of possible solutions in order to meet business criteria. The first step in creating the constraints is to state the business objects in words and then translate those into mathematical formulas. Our blend model must meet the following business objectives: 

1. the total amount of blended crude must be no greater than the tank capacity
2. we want to know the per barrel profit *and loss* from blending
3. the blended grade must meet the required quality specifications
4. in practice there must never be more than three grades in a single blend

### 1. Tank Capacity Constraint

The amount of blended crude cannot exceed the tank capacity $T$. Since the total volume of the blend is simply the sum of the component volumes, this constraint can be expressed as 

$$
\sum_{i \in C} x_i \leq T
$$

### 2. Computing the Profit (or Loss) from Blending

We are interested in the profit on the optimal blend, *even if the profit is negative (a loss)*. This is important for seeing how close conditions are to making blending profitable. Recall that we imposed the constraint that $x_i \geq 0$. If there is no profitable blend, the model will set $x_i = 0$ for all $i$ unless we impose the constraint that the model is required to blend a non-zero amount of crude. The constaint amounts to setting a lower bound on the total blended volume. The number can be an arbitrary positive number but in this example we set it to be 1 because it will make calculation of the the per-barrel loss easy when there is exactly one barrel in the blend. 

$$
\sum_{i \in C} x_i > 1
$$

The combined constraint is

In [ ]:
## total constraint 

def total_barrel_constraint(model):
    min_ = 1.0
    return (min_, sum(model.x[i] for i in model.CRUDES), constraints['total'])

model.total_barrel_constraint = pyo.Constraint(rule=total_barrel_constraint)

### 3. Quality Constraints

We assume that all qualities blend linearly, or that they have been converted to blend indices which do. A quality blends linearly if it can expressed as a weighted average. Let $L_j$ and $U_j$ be the upper and lower bounds on the $j$-th quality constraint and let $q_{ij}$ be the $j$-th quality of the $i$-th crude.

#### Volume-Based Qualities
Most of the 


#### Weight-Based Qualities

In [ ]:
_volume_constraints = ['rvp_idx', 'sg', 'pour_idx']
def quality_constraint_upper_bound(model, q):
    if q in _volume_constraints:
        return sum(model.x[c] * (model.qualities[c, q] - constraints['quality'][q][1]) for c in model.CRUDES) <= 0
    else:
        return sum(model.qualities[c, "sg"] * model.x[c] * (model.qualities[c, q] - constraints['quality'][q][1]) for c in model.CRUDES) <= 0
    
def quality_constraint_lower_bound(model, q):
    if q in _volume_constraints:
        return sum(model.x[c] * (model.q[c, q] - constraints['quality'][q][0]) for c in model.CRUDES) >= 0
    else:
        return sum(model.qualities[c, "sg"] * model.x[c] * (model.qualities[c, q] - constraints['quality'][q][0]) for c in model.CRUDES) >= 0
    
model.quality_constraint_ub = pyo.Constraint(model.QUALITIES, rule=quality_constraint_upper_bound)
model.quality_constraint_lb = pyo.Constraint(model.QUALITIES, rule=quality_constraint_lower_bound)

In [ ]:
model.xi = pyo.Var(model.CRUDES, domain=pyo.Binary)

def max_crudes_constraint(model, n_max):
    return sum(model.xi[c] for c in model.CRUDES) <= n_max

model.max_crudes_constraint = pyo.Constraint(rule=max_crudes_constraint)

In [ ]:
def binary_constraint(model, c):
    return model.x[c] - constraints['supply'][c][1] * model.xi[c] <= 0

model.binary_constraint = pyo.Constraint(model.CRUDES, rule=binary_constraint)

In [ ]:
def profit_maximization(model):
    return sum(model.prices[c] * model.x[c] for c in model.CRUDES)

model.objective = pyo.Objective(rule=profit_maximization, sense=pyo.maximize)